# Wikipedia Notable Life Expectancies
# [Notebook 7 : Data Cleaning Part 6](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean6_thanak_2022_07_24.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean5.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean5", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98060 rows and 38 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,economist,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None,None,2.564949,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98058,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",None,,MNA,None,None,None,None,None,None,None,None,50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,0,0,0,0,0,1,0,0,1,0,0,0,2
98059,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,engineer,member of the Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None,None,1.386294,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
84934,29,Valeriy Babych,", 67, Ukrainian politician, Deputy , economist and businessman, COVID-19.",https://en.wikipedia.org/wiki/Valeriy_Babych,16,2020,October,,None,,Deputy,economist and businessman,COVID,None,None,None,None,None,None,67.0,None,Ukraine,None,1994 2002,2.833213,0,0,0,0,0,0,0,0,1,0,0,0,1
40911,3,"Peter Johnson, Sr.",", 91, American trial lawyer and political power broker, pulmonary fibrosis.","https://en.wikipedia.org/wiki/Peter_Johnson,_Sr.",3,2012,December,None,None,trial lawyer and political power broker,pulmonary fibrosis,None,None,None,None,None,None,None,None,91.0,None,United States of America,None,None,1.386294,0,0,0,0,0,0,0,0,0,0,0,0,0
29951,10,Dilip Chitre,", 70, Indian poet, cancer.",https://en.wikipedia.org/wiki/Dilip_Chitre,3,2009,December,None,None,poet,cancer,None,None,None,None,None,None,None,None,70.0,None,India,None,None,1.386294,0,0,0,0,0,0,0,0,0,0,0,0,0
65585,6,Gao Mang,", 90–91, Chinese translator.",https://en.wikipedia.org/wiki/Gao_Mang,12,2017,October,None,None,translator,None,None,None,None,None,None,None,None,None,90.5,None,"China, People's Republic of",None,None,2.564949,0,0,0,0,0,0,0,0,0,0,0,0,0
44442,13,Damon Intrabartolo,", 39, American playwright .",https://en.wikipedia.org/wiki/Damon_Intrabartolo,7,2013,August,"and orchestrator ,",None,playwright,None,None,None,None,None,None,None,None,None,39.0,None,United States of America,None,"and orchestrator ,",2.079442,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98060 entries, 0 to 98059
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98060 non-null  object 
 1   name                       98060 non-null  object 
 2   info                       98060 non-null  object 
 3   link                       98060 non-null  object 
 4   num_references             98060 non-null  int64  
 5   year                       98060 non-null  int64  
 6   month                      98060 non-null  object 
 7   info_parenth               36661 non-null  object 
 8   info_1                     22 non-null     object 
 9   info_2                     98028 non-null  object 
 10  info_3                     48896 non-null  object 
 11  info_4                     10264 non-null  object 
 12  info_5                     1265 non-null   object 
 13  info_6                     181 non-null    obj

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting known_for values by creating `known_for_dict_14`.

### Extracting `known_for` Continued

#### Finding `known_for` Roles in `info_2`

In [6]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [359]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [265]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[[index for index in df.index if "poet" in df.loc[index, "info"]], "info_2",]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [361]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [263]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "poet and poetry" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [262]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "poet and fy worker"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [261]:
# Creating lists for each category
politics_govt_law = []

arts = [
    "Pulitzer Prize winning poet and former poet laureate",
    "poet of the School",
    "poet and dramatist",
    "Nuyorican poet and playwright",
    "poet and memoirist",
    "poet and BBC producer",
    "Mi'kmaq poet",
    "poet and Pulitzer Prize winner",
    "poet and arts critic",
    "poet and radio host",
    "novelist and poetry promoter",
    "poet and diarist",
    "poet and broadcaster",
    "surrealist poet",
    "Chicano poet",
    "Martiniquan poet",
    "Māori poet",
    "poet who wrote about the Dust Bowl",
    "vocalist and poet",
    "satirist and humorist poet of",
    "Movement poet",
    "avant garde poet and visual artist",
    "poet and literary book publisher",
    "poet of the Beat Generation",
    "Latino poet",
    "contemporary poet",
    "film maker and poet",
    "beat poet",
    "biographer and poet",
    "poet and jazz musician",
    "poet and architecture critic",
    "K'iche' Maya poet",
    "visual artist and poet",
    "poet and biographer",
    "jazz pianist and poet",
    "rhythm poet and musician",
    "dub poet",
    "percussionist and poet",
    "poet and jazz pianist",
    "poet and spoken word musician",
    "poet and sculptor",
    "experimental poet",
    "poet and radio broadcaster",
    "Marathi ghazal poet",
    "poet and co founder of interstitial lung disease",
    "avant garde composer and poet",
    "magazine publisher and poet",
    "literary critic and poet",
    "sculptor and poet",
    "poet and proofreader",
    "Odia poet",
    "folk musician and poet",
    "Ulster Scots poet",
    "vernacular poet",
    "Pashto poet",
    "poet and recording artist",
    "poet and disc jockey",
    "Kannada language poet",
    "and spoken word poet",
    "poet and film producer",
    "Nuyorican poet",
    "Kannada poet",
    "surrealist poet and art critic",
    "director and poet",
    "Kashubian poet",
    "poet and poetry",
    "art critic and poet",
    "poet and art critic",
    "jazz musician and poet",
    "Native poet",
    "poet and composer",
    "poet and cartoonist",
    "poet and filmmaker",
    "Pulitzer Prize winning poet",
    "photographer and poet",
    "poet and visual artist",
    "musician and poet",
    "poet and performance artist",
    "poet and musician",
    "lyricist and poet",
    "painter and poet",
    "poet and artist",
    "poet and publisher",
    "playwright and poet",
    "poet and painter",
    "poet and lyricist",
    "Urdu poet and",
    "Urdu poet",
    "poet and essayist",
    "artist and poet",
    "poet and critic",
    "poet and editor",
    "poet and literary critic",
    "novelist and poet",
    "poet and playwright",
    "poet and novelist",
    "Occitan language poet and",
    "language poet",
    "Arabian poet and",
    "poetess",
    "poet and literary",
    "Beat generation poet and",
    "Beat Generation poet",
    "Beat poet",
    "prize winning poet and",
    "n poet",
    "poet laureate",
    "poet and",
    "and poet",
    "poets",
    "poet",
]
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

In [362]:
# Hard-coding cause_of_death value found in info_2
index = df[df["link"] == "https://en.wikipedia.org/wiki/Lawrence_Ferlinghetti"].index
df.loc[index, "cause_of_death"] = "interstitial lung disease"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict_14` Dictionary of Category Keys and Specific Role Lists of Values

In [363]:
# Combining separate lists into one dictionary
known_for_dict_14 = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [364]:
%%time

# Dictionary version
search_dict = known_for_dict_14

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

CPU times: total: 1min 3s
Wall time: 1min 3s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
95191,9,Akira Inoue,", 93, Japanese film director , stroke and pneumonia.",https://en.wikipedia.org/wiki/Akira_Inoue_(film_director),8,2022,January,", ,",None,,stroke and pneumonia,None,None,None,None,None,None,None,None,93.0,None,Japan,None,", ,",2.197225,0,0,0,0,0,1,0,0,0,0,0,0,1
20358,27,Ronald Pearsall,", 77, English author.",https://en.wikipedia.org/wiki/Ronald_Pearsall,4,2005,September,None,None,,None,None,None,None,None,None,None,None,None,77.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,1.609438,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [365]:
#### Checking the number of rows without a first category
df["num_categories"] = df[known_for_dict_14.keys()].sum(axis=1)

print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 53857 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to build `known_for_dict_15` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [367]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [985]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [747]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "artist" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [983]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [892]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "artist and art" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [984]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "martial artist and teacher"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [989]:
# Creating lists for each category
politics_govt_law = []

arts = [
    "based graffiti artist whose works were included in the Venice Biennale",
    "game artist",
    "Oscar winning make up artist",
    "choreographer appointed as Sydney Dance Company artistic director",
    "based pop artist",
    "artist and wife of Joaquín Torres García",
    "botanical artist",
    "television make up artist",
    "artist and industrial designer",
    "movie poster artist",
    "courtroom artist",
    "artist and engraver",
    "graphic designer and psychedelic artist",
    "special effects and make up artist",
    "comic book artist and publisher",
    "artist and theatre designer",
    "pop artist and director",
    "artistic draughtsman",
    "artist known for his role in the Conceptualism and Minimalism movements",
    "aboriginal artist",
    "native artist",
    "musician and artist",
    "ballet dancer and artistic director of Ballet",
    "master potter and artist",
    "drag queen music artist",
    "internationally recognized graphic artist",
    "fantasy and science fiction artist and illustrator",
    "internationally exhibited Navajo artist",
    "comic book artist and co founder of",
    "artist who was a member of the Regina Five",
    "animator and layout artist",
    "colorist and cover production artist for DC Comics",
    "special effects artist and pedal steel guitarist",
    "comic book colourist and artist",
    "based architect and artist",
    "installation artist and assemblage sculptor",
    "artist and television presenter",
    "movie artist and illustrator",
    "artist and watercolourist",
    "naïve artist",
    "transgender artist",
    "special effects artist and producer",
    "artist and musical performer",
    "and butter sculpture artist",
    "artist and print maker",
    "pop artist and sculptor",
    "experimental music artist",
    "blues musician and artist",
    "comic book artist and reputed creator of",
    "artist and composer",
    "ceramic artist and designer",
    "video game concept artist",
    "artist and reporter",
    "performance artist and playwright",
    "artist and muralist",
    "animation artist and character designer",
    "blues artist",
    "Academy Award winning visual effects artist",
    "vocalist and bassist and solo artist",
    "artist of origin",
    "neo mannerist artist",
    "shadow play artist",
    "sculptor and conceptual artist",
    "film make up artist",
    "tapestry and textile artist",
    "Golden Age comic book artist",
    "rock musician and artist",
    "strip artist",
    "wood carving artist",
    "plastic artist and illustrator",
    "artist and banknote designer",
    "textile artist and printmaker",
    "fantasy and science fiction artist",
    "figurative expressionist artist",
    "hip hop musician and graffiti artist",
    "Navajo artist",
    "wet folding origami artist",
    "Route artist",
    "animator and comic book artist",
    "R&B artist",
    "mural artist",
    "makeup artist and tenor",
    "artist and lecturer",
    "artist and ceramicist",
    "experimental visual artist",
    "vocalist and session artist",
    "artists' model and memoirist",
    "comic book and comic strip artist",
    "comic strip artist and editor",
    "Native artist and potter",
    "West Coast artist",
    "artist and comic book creator",
    "graphic artist and sculptor",
    "painter and visual artist",
    "New Realist artist",
    "motion picture matte artist",
    "artist and novelist",
    "abstract and representational artist",
    "broadcaster and comic book artist",
    "neo conceptual artist",
    "painter and pioneering manhua artist",
    "hillbilly and bluegrass artist",
    "artist and member of the Fluxus movement",
    "artist and member of the Ultra Lettrist movement",
    "graphic artist and printmaker",
    "musician and recording artist",
    "textile artist who specialized in embroidery",
    "ink artist and wife of Walt Disney",
    "artist and art collector",
    "n artist from Utopia",
    "Chicano artist",
    "artist of origins",
    "artist and doll maker",
    "conceptual and performance artist",
    "studio potter and ceramic artist",
    "tenor and artist",
    "country music artist",
    "visual artist known for her still lives and landscapes",
    "film and video artist",
    "artist and watercolor master",
    "painter and comics artist",
    "vocalist and recording artist",
    "comic book artist born",
    "science fiction and fantasy artist",
    "novelist and artist",
    "conductor and recording artist",
    "botanical artist and art critic",
    "carving artist",
    "visual artist and conceptual sculptor",
    "contemperary artist",
    'sculptor and "one of the nation most accomplished medallic artists"',
    "cartoonist and comic artist",
    "rock artist",
    "psychedelic artist",
    "animation and comic book artist",
    "textile artist and embroiderer",
    "artist and experimental photographer",
    "cartoonist and comic book artist",
    "potter and ceramic artist",
    "artist anddesigner",
    "born artist",
    "sculptor and stained glass artist",
    "equine artist",
    "typographer and graphic artist",
    "visual artist and jewelry and fashion designer",
    "visual artist and protégé of Salvador Dalí",
    "reggae artist",
    "artist and Army art correspondent",
    "film special effects artist",
    "vocalist and artist",
    "experimental filmmaker and artist",
    "fine artist and art editor",
    "comic book artist and book illustrator",
    "Les Automatistes artist and a member of",
    "comic book artist and co creator of Jonah Hex and Black Orchid",
    "Papunya Tula artist",
    "graphic artist and postage stamp designer",
    "artist and metalsmith",
    "manhua artist",
    "illustrator and storyboard artist",
    "Western artist",
    "artist and architectural designer",
    "graphic artist and banknote designer",
    "artist in wood",
    "Yakshagana artist",
    "trapeze artist",
    "newspaper artist and cartoonist",
    "artist and puppeteer",
    "ceramic artist and sculptor",
    "furniture designer and artist",
    "rock album cover artist",
    "stateless auto destructive artist",
    "indigenous artist and printmaker",
    "computer artist",
    "watercolour artist",
    "media artist and designer",
    "artist and landscape architect",
    "Eurodance artist",
    "musician and comic book artist",
    "graphic artist and game designer",
    "comics artist and graphic novelist",
    "hologram artist",
    "cartoonist and comics artist",
    "comics artist and animator",
    "reggae artist and comedian",
    "sound installation artist and musician",
    "artist and potter",
    "artist and film production illustrator",
    "multimedia artist and painter",
    "film animator",
    "artist photographer",
    "experimental filmmaker and visual artist",
    "artist and jewelry designer",
    "graphic designer and film poster artist",
    "fetish artist",
    "avant garde installation artist and sculptor",
    "courtroom sketch artist",
    "sketch artist",
    "caricaturist and comics artist",
    "Iñupiat artist",
    "hand shadow artist",
    "Indigenous artist",
    "musical Thavil artist",
    "ceramist and textile artist",
    "radio presenter and artist",
    "comics artist and cartoonist",
    "lianhuanhua artist",
    "multi media artist",
    "animator and comics artist",
    "artist and cultural figure",
    "typographer and visual artist",
    "graphic designer and album artist",
    "pencil artist",
    "theatre artist and playwright",
    "comic artist and illustrator",
    "motorsports artist",
    "voice over and recording artist",
    "bassist and artist",
    "comic creator and cover artist",
    "tattoo artist and reality show personality",
    "cartoonist and street artist",
    "sand artist",
    "architect and light artist",
    "media producer and makeup artist",
    "Kiowa artist",
    "Peking opera artist",
    "electronic music artist and MC",
    "graphic design artist",
    "impressionist artist",
    "graphic designer and street artist",
    "illustrator of children books and cartoon artist",
    "artist and animator",
    "record producer and artist",
    "rock concert graphic poster artist",
    "land artist",
    "video and visualization artist",
    "artist and weaver",
    "painter and plastic artist",
    "fantasy gaming artist",
    "Madhubani painter and artist",
    "animation director and storyboard artist",
    "conceptual artist and photographer",
    "fantasy artist and album cover designer",
    "design artist and painter",
    "theatre artiste",
    "thangka artist",
    "artist and Alghoza player",
    "fantasy coffin artist",
    "film concept artist",
    "wildlife artist and illustrator",
    "theatre personality and artist",
    "visual artist and fashion designer",
    "artist and fashion designer",
    "comedic artist",
    "sculptor and artistic director",
    "graphic designer and album cover artist",
    "artist and trading card illustrator",
    "geometric artist",
    "founder and artistic director of the Melbourne Theatre Company",
    "indigenous artist",
    "artist and book cover illustrator",
    "punk graphic designer and artistic director",
    "television host and recording artist",
    "artistic director and live performance organizer",
    "weaving artist",
    "pop and minimalist artist",
    "dubbing artist",
    "monumentalist artist",
    "sound artist and electronic music composer",
    "highwire artiste",
    "pixel artist",
    "conceptual and digital artist",
    "artist and inventor of the plastic pink flamingo",
    "batik artist",
    "enamel artist",
    "operatic tenor and artistic director",
    "sound artist and radio presenter",
    "sound artist",
    "conductor and artistic director",
    "graphic artist and designer",
    "quilt artist",
    "comic strip artist and cartoonist",
    "painter and installation artist",
    "graphic artist and cartoonist",
    "recording artist and vocalist",
    "circus artist and animal trainer",
    "Kunqu artist",
    "surrealist artist",
    "harmonism artist",
    "graphic designer and poster artist",
    "marine artist",
    "commercial artist and illustrator",
    "commercial artist",
    "plastic artist",
    "color abstract artist",
    "special effects make up artist",
    "underground graffiti artist",
    "ballet master and artistic director",
    "animation background artist",
    "hymnist and visual artist",
    "recording artist",
    "visual artist and photographer",
    "World War II artist",
    "painter and kinetic artist",
    "muralist and pictorial artist",
    "cartoonist and artist",
    "sculptor and graphic artist",
    "visual artist and musician",
    "artists' model",
    "abstract expressionist artist and",
    "abstract expressionist artist",
    "artist and printmaker",
    "sculptor and installation artist",
    "installation artist",
    "filmmaker and artist",
    "comic book artist and painter",
    "sculptor and performance artist",
    "artist and set designer",
    "comic book artist and editor",
    "cabaret artist",
    "environmental artist",
    "expressionist artist",
    "comic book and advertising artist",
    "kinetic artist",
    "concept artist",
    "fantasy artist",
    "heraldic artist",
    "Ojibwe artist",
    "First Nations artist",
    "ballet dancer and artistic director",
    "mime artist",
    "contemporary visual artist",
    "psychedelic poster artist",
    "tattoo artist",
    "artist and art critic",
    "hip hop artist",
    "forensic artist",
    "illustrator and comics artist",
    "Hall of Fame comic book artist",
    "Inuk artist",
    "figurative artist",
    "folk artist",
    "artist and filmmaker",
    "voice over artist",
    "minimalist artist",
    "artist and graphic designer",
    "nonconformist artist",
    "portrait artist",
    "pottery artist",
    "film poster artist",
    "fiber artist",
    "Kathakali artist",
    "illustrator and comic book artist",
    "science fiction artist",
    "watercolor artist",
    "comic book artist and cartoonist",
    "artist and cartoonist",
    "avant garde artist",
    "realist painter and graphic artist",
    "comics artist and illustrator",
    "ceramics artist",
    "outsider artist",
    "architect and artist",
    "street artist",
    "video artist and",
    "video artist",
    "contemporary artist",
    "Inuit artist",
    "painter and artist",
    "voice artist",
    "rap artist",
    "graffiti artist",
    "special effects artist",
    "visual effects artist and",
    "visual effects artist",
    "and landscape artist",
    "landscape artist",
    "stained glass artist",
    "graphic designer and artist",
    "designer and artist",
    "war artist",
    "comic strip artist",
    "artistic director",
    "pop artist",
    "make up artist and",
    "make up artist",
    "comic artist",
    "wildlife artist",
    "artist and photographer",
    "sculptor and artist",
    "artist and musician",
    "makeup artist",
    "artist and architect",
    "ceramic artist and",
    "ceramic artist",
    "glass artist",
    "textile artist",
    "artist and designer",
    "artist and painter",
    "artist and sculptor",
    "performance artist and",
    "and performance artist",
    "performance artist",
    "artist and illustrator",
    "painter and graphic artist",
    "and graphic artist",
    "graphic artist and",
    "graphic artist",
    "conceptual artist",
    "abstract artist",
    "manga artist and",
    "manga artist",
    "comics artist and",
    "and comics artist",
    "comics artist",
    "visual artist and art",
    "and visual artist",
    "visual artist and",
    "visual artist",
    "comic book artist and",
    "comic book artist",
    "woodcut artist and",
    "poster artist and",
    "poster artist",
    "and storyboard artist",
    "storyboard artist",
    "modern artist and",
    "and fish skin artist",
    "and artist",
    "artist and",
    "n artist",
    'music artist',
    "artist",
]
sports = [
    "Muay martial artist",  # before arts
    "martial artist and Isshinryu karate pioneer",
    "professional wrestler and mixed martial artist",
    "martial artist and founder of Modern Arnis",
    "Olympic artistic gymnast",
    "professional wrestler and martial artist",
    "artistic gymnastics coach",
    "mixed martial artist and kickboxer",
    "artistic gymnast and Olympian",
    "mixed martial artist and grappler",
    "super heavyweight kickboxing champion and mixed martial artist",
    "Olympic taekwondo martial artist",
    "mixed martial artist and professional wrestler",
    "martial artist and teacher",
    "kickboxer and mixed martial artist",
    "artistic gymnast",
    "mixed martial artist",
    "martial artist and",
    "and martial artist",
    "martial artist",
]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = [
    "first civilian to receive the Intelligence Medal of Merit"
]
spiritual = ["modern primitive proponent"]
social = []
crime = ["scam artist", "con artist and", "art forger"]  # before arts
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict_15` Dictionary of Category Keys and Specific Role Lists of Values

In [990]:
# Combining separate lists into one dictionary
known_for_dict_15 = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "sports": sports,
    "arts": arts,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [991]:
%%time

# Dictionary version
search_dict = known_for_dict_15

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

CPU times: total: 5min 1s
Wall time: 5min 1s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
70631,3,Jacqueline Pearce,", 74, British actress , lung cancer.",https://en.wikipedia.org/wiki/Jacqueline_Pearce,49,2018,September,", ,",None,,lung cancer,None,None,None,None,None,None,None,None,74.0,None,United Kingdom of Great Britain and Northern Ireland,None,", ,",3.912023,0,0,0,0,0,1,0,0,0,0,0,0,1
8695,6,Antonio Pierfederici,", 79, Italian actor.",https://en.wikipedia.org/wiki/Antonio_Pierfederici,3,1999,January,None,None,,None,None,None,None,None,None,None,None,None,79.0,None,Italy,None,None,1.386294,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [992]:
#### Checking the number of rows without a first category
df["num_categories"] = df[known_for_dict_15.keys()].sum(axis=1)

print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 51956 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to build `known_for_dict_16` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [994]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [1183]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [1182]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "painter" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [1179]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [1180]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "provocateur" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [1181]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "painter and chess composer"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [1186]:
# Creating lists for each category
politics_govt_law = [
    "provocateur",
]

arts = [
    "painter and drawer",
    "Native painter",
    "rock musician and painter",
    "naïve painter",
    "Puerto Ricanpainter and art",
    "playwright and painter",
    "born painter",
    "architect and painter",
    "engraver and painter",
    "outback painter",
    "Cubist painter",
    "cityscape painter",
    "painter and Cultural Medallion winner",
    "film critic and painter",
    "art critic and surrealist painter",
    "painter of mystical Jewish works",
    "color field painter",
    "painter and lecturer",
    "Andalusian painter and illustrator",
    "painter and glass sculptor",
    "aboriginal painter",
    "Delftware painter",
    "barn painter",
    "abstract painter and printmaker",
    "figurative painter and arts",
    "television painter",
    "draughtsman and painter",
    "mixed media painter",
    "enameller and painter",
    "landscape painter and watercolorist",
    "painter and watercolorist",
    "painter and television host",
    "folk art painter",
    "portrait painter and sculptor",
    "Realist painter",
    "Yōga painter",
    "photographer and portrait painter",
    "portrait and landscape painter",
    "figurative painter and draftsman",
    "painter and protégé of Pablo Picasso",
    "painter and calligrapher",
    "painter and ceramicist",
    "painter and scenographer",
    "watercolour painter",
    "painter and draftsman",
    "animator and painter",
    "painter and weaver",
    "painter and lithographer",
    "painter and cartoonist",
    "Ojibwe painter",
    "cartoonist and painter",
    "stage designer and painter",
    "surrealistic painter",
    "painter and cinematographer",
    "painter and draughtsman",
    "painter and theatre director",
    "icon painter",
    "Coptic painter",
    "fashion designer and painter",
    "designer and painter",
    "composer and painter",
    "signpainter",
    "painter and television presenter",
    "representational painter",
    "painter and print maker",
    "figurative painter",
    "painter and animator",
    "abstract painter and print maker",
    "painter and composer",
    "painter and children book illustrator",
    "art critic and painter",
    "painter of themes",
    "yodeler and painter",
    "geometric abstractionist painter",
    "scenographer and painter",
    "impressionist painter",
    "Yolngu painter",
    "painter and ceramist",
    "magic realist painter",
    "painter and dancer",
    "photorealist painter",
    "watercolor painter",
    "jazz musician and painter",
    "musician and painter",
    "painter and graphic designer",
    "modernist painter",
    "muralist and painter",
    "avant garde painter",
    "COBRA painter",
    "painter and muralist",
    "abstract painter and sculptor",
    "painter and novelist",
    "painter and musician",
    "photographer and painter",
    "portrait painter",
    "painter and designer",
    "and realist painter",
    "realist painter and",
    "surrealist painter",
    "realist painter",
    "illustrator and painter",
    "abstract expressionist painter",
    "expressionist painter",
    "landscape painter",
    "painter and engraver",
    "painter and photographer",
    "painter and illustrator",
    "abstract painter",
    "sculptor and painter",
    "painter and printmaker",
    "painter and sculptor",
    "painter and",
    "and painter",
    "painter",
]
sports = ["chess composer"]
sciences = []

business_farming = []
academia_humanities = ["theatre school director"]
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict_7` Dictionary of Category Keys and Specific Role Lists of Values

In [1187]:
# Combining separate lists into one dictionary
known_for_dict_16 = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [1188]:
%%time

# Dictionary version
search_dict = known_for_dict_16

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

CPU times: total: 1min 23s
Wall time: 1min 23s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
51926,6,Carl Cunningham-Cole,", 72, British ceramic artist.",https://en.wikipedia.org/wiki/Carl_Cunningham-Cole,3,2015,February,None,None,,None,None,None,None,None,None,None,None,None,72.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,1.386294,0,0,0,0,0,1,0,0,0,0,0,0,1
19152,22,Consuelo Velázquez,", 88, Mexican songwriter and lyricist, and author of the enduring song ""Bésame mucho"".",https://en.wikipedia.org/wiki/Consuelo_Vel%C3%A1zquez,12,2005,January,None,None,song,"and author of the enduring song ""Bésame mucho""",None,None,None,None,None,None,None,None,88.0,None,Mexico,None,None,2.564949,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [1189]:
#### Checking the number of rows without a first category
df["num_categories"] = df[known_for_dict_16.keys()].sum(axis=1)

print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 51002 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to build `known_for_dict_17` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [1191]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [1582]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [1578]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "musician" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [1581]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [1576]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "Shinshu" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [1575]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "stalker of musician Björk"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [1574]:
# Creating lists for each category
politics_govt_law = []

arts = [
    "musician known for his work with the Beatles",
    "musician and host of",
    "musician with the Bothy Band",
    "award winning blues musician",
    "shehnai musician and Bharat Ratna winner",
    "musician known for his work on the theme",
    "musician who played at the Johannesburg Live concert",
    "jazz musician and widow of John Coltrane",
    "musician and husband of Celia Cruz",
    "musician and founding conductor of Brooklyn Philharmonic Orchestra",
    "musician and entertainer",
    'musician known as "Saint Thomas"',
    "theme song composer and jazz musician",
    "composer and jazz musician",
    "Benga musician",
    "musician credited with creating mambo",
    "saxophone and clarinet musician",
    "musician and composer from the band Savage Rose",
    "musician and founder of the band Bathory",
    "musician and playwright",
    "Celtic musician",
    "Rockabilly musician",
    "R&B and country & western musician",
    "Tony nominated orchestrator and musician",
    "fiddle player and bluegrass musician",
    "musician with the hardcore punk band Big Boys",
    "musician and founding member of The Vandals",
    "musician for acoustic rock band Plush",
    "steel pan musician",
    "musician and bagpiper",
    "musician and essayist with the stage name Buddy Blue",
    "vocal jazz musician",
    "dancehall garage musician",
    "Romani musician",
    "cartoonist and musician",
    "soca musician",
    "musician and one man band",
    "cajun musician",
    "rapper and musician",
    "country musician and producer",
    "soul and disco musician",
    "street musician",
    "Grammy Award winning jazz producer",
    "Indorock musician",
    "musician and festival organiser",
    "comedian and musician",
    "musician and band manager",
    "jazz musician and jazz critic",
    "traditional jazz musician",
    "country musician and trumpet player",
    "soul and funk musician",
    "country musician and comedian",
    "musician and comedian",
    "hip hop DJ and musician",
    "musician and patron of the arts",
    "club disc jockey and musician",
    "Reggae musician and composer",
    "musician and hula expert",
    "jazz and rock musician",
    "garage punk musician",
    "jazz musician and accordionist",
    "musician and sound recordist",
    "jazz guitarist and studio musician",
    "Classical musician",
    "country musician and television and radio host",
    "keyboardist and session musician",
    "musician and radio and TV personality",
    "Country & Western musician",
    "rock and R&B musician",
    "R&B and jazz musician",
    "rock 'n roll musician",
    "musician and music business executive",
    "country music and rockabilly musician",
    "dixieland jazz musician",
    "guitarist and studio musician",
    "musician and inventor of board game Cluedo",
    "classical musician and tabla player",
    "soul musician and a guitarist",
    "reggae musician and Rastafarian",
    "concert and easy listening musician",
    "blues and folk musician",
    "Jùjú musician",
    "country blues musician",
    "musician and music publisher",
    "kadongo kamu musician",
    "reggae musician and producer",
    "musician and creator of the seggae genre",
    'jazz musician known as "Mr Swing"',
    "musician of music",
    "children musician",
    "musician and DJ of Run DMC",
    "skiffle musician",
    "musician and star of the Buena Vista Social Club",
    "string band fiddler and mandolinist and country blues musician",
    "musician composer and arranger",
    "rockabilly composer and musician",
    "soul musician and brother of Marvin Gaye",
    "musician and serial",
    "Volksmusik musician and collector",
    "musician and pioneer of the musical genres of and",
    "bluegrass and folk musician",
    "bandleader and musician",
    "big band style musician",
    "Jibaro musician",
    "folk musician and composer",
    "musician and vocal session arranger",
    "musician and composer of electronic music",
    "musician and former member of The Beatles",
    "Tejano musician",
    "jazz fusion musician",
    "entertainer and musician",
    "outsider musician",
    "trumpeter and session musician",
    "architect and musician",
    "electronic musician and radio host",
    "sound designer and musician",
    "electronic dance musician and record producer",
    "jazz musician and arranger",
    "Nazrul Sangeet musician",
    "musician and publisher",
    "Indigenous musician",
    "folk and bluegrass musician",
    "punk and new wave musician",
    "trance music producer and musician",
    "Carnatic musician and composer",
    "Hindustani musician",
    "jazz musician and band leader",
    "shehnai musician",
    "rap musician",
    "avant garde musician",
    "ska and mento musician",
    "synthesizer musician",
    "musician and choral conductor",
    "samba musician",
    "khyal musician",
    "kwaito musician",
    "acid house musician",
    "photographer and musician",
    "minimalist musician",
    "trumpeter and brass band musician",
    "soundtrack composer and musician",
    "grime musician",
    "folk rock musician and composer",
    "disc jockey and musician",
    "funk and R&B musician",
    "Grammy award winning musician",
    "musician and organist",
    "blues and rock musician",
    "musician and radio presenter",
    "Hall of Fame instrumental and surf rock musician",
    "surf rock musician",
    "Bubu musician",
    "musician and dancer",
    "Andean cumbia musician",
    "ashik musician",
    "Hall of Fame blues musician",
    "sculptor and musician",
    "experimental musician",
    "jazz musician and Shinshu Buddhist",
    "mbira musician",
    "theatre director and musician",
    "jazz musician and producer",
    "Steelpan musician and arranger",
    "country musician and radio broadcaster",
    "musician and arranger",
    "punk rock musician and",
    "punk rock musician",
    "carnatic musician and music director",
    "musician and music director",
    "steelpan musician and designer",
    "steelpan musician",
    "Tuvan musician",
    "bossa nova musician",
    "vallenato musician",
    "music manager and musician",
    "contra dance musician",
    "bluegrass musician and banjo player",
    "country and rockabilly musician",
    "roots musician and entertainment critic",
    "roots musician",
    "jazz musician and manager",
    "jazz musician and radio show host",
    "electronic musician and television producer",
    "jazz musician and vocal coach involved in the Wrong Door Raid",
    "Detroit blues musician",
    "folk musician and yueqin player",
    "blues and country musician",
    "heavy metal musician",
    "rock and roll session musician",
    "jazz musician and architect",
    "musician and radio and television personality",
    "Hall of Fame bluegrass musician",
    "oud musician",
    "music  musician",
    "cumbia musician",
    "book and album cover designer and jazz musician",
    "salsa musician and composer",
    "musician and percussion mallet manufacturer",
    "Gnawa musician",
    "blues rock musician",
    "musician and music publishing executive",
    "musician and television show host",
    "composer and electronic musician",
    "Māori musician",
    "and Manager of Jazz musician Erroll Garner",
    "musician and house music producer",
    "mariachi musician and",
    "mariachi musician",
    "merengue and salsa musician",
    "pop musician and producer",
    "reggae cross over musician",
    "Grammy Award winning jazz and new age musician",
    "classical dancer and musician",
    "dancer and musician",
    "musician and film composer",
    "rhythm and blues musician",
    "jazz and pop musician",
    "musician and music producer",
    "electric blues musician",
    "musician and DJ",
    "tango musician",
    "Igbo highlife musician",
    "dancehall musician",
    "ska musician",
    "calypso musician",
    "soul musician",
    "session musician",
    "ambient musician",
    "Chicago blues musician",
    "Hall of Fame musician and record producer",
    "big band musician",
    "electronic musician and composer",
    "folk rock musician",
    "Cajun musician",
    "Grammy Award winning musician",
    "jazz and blues musician",
    "traditional musician",
    "reggae musician and record producer",
    "jazz musician and bandleader",
    "musician and bandleader",
    "pop musician",
    "polka musician",
    "musician and conductor",
    "Carnatic musician",
    "punk musician",
    "broadcaster and musician",
    "rock and roll musician",
    "R&B musician",
    "highlife musician",
    "hip hop musician",
    "record producer and musician",
    "musician and producer",
    "classical musician and",
    "classical musician",
    "Hall of Fame musician",
    "rockabilly musician",
    "electronic musician",
    "jazz musician and composer",
    "reggae musician",
    "musician and record producer",
    "country musician",
    "and bluegrass musician",
    "bluegrass musician",
    "composer and musician",
    "musician and composer",
    "and traditional folk musician",
    "folk musician and",
    "and folk musician",
    "folk musician",
    "and rock musician",
    "rock musician",
    "blues musician and",
    "blues musician",
    "and jazz musician",
    "jazz musician",
    "benga musician and",
    "musician and television",
    "Native musician and",
    "musician from",
    "and musician",
    "musician and",
    "musician",
]
sports = []
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = [
    "stalker of musician Björk",  # before arts
]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict_17` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict_17 = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict_17

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
#### Checking the number of rows without a first category
df["num_categories"] = df[known_for_dict_17.keys()].sum(axis=1)

print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to build `known_for_dict_18` for the next iteration.

In [1190]:
print("dunzo!")

# Sound notification when cell executes
chime.success()

dunzo!


<IPython.core.display.Javascript object>

#### Finding `known_for` Roles in `info_2`

In [ ]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

In [ ]:
# Code to check each value
roles_list.pop()

In [ ]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "writer" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

In [ ]:
# # Code to check each specific value
# specific_roles_list.pop()

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "and science writer" in df.loc[index, "info"]]]

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [ ]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "outlaw country music singer songwriter"]

#### Creating Lists for Each `known_for` Category

In [ ]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [


]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

#### Creating `known_for_dict_7` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict_7 = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict_7

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
#### Checking the number of rows without a first category
df["num_categories"] = df[known_for_dict_7.keys()].sum(axis=1)

print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to build `known_for_dict_8` for the next iteration.